# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [52]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [53]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [54]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [55]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [56]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [57]:
try: 
    session.execute(""" CREATE  KEYSPACE IF NOT EXISTS music_library_db 
                               WITH REPLICATION = { 
                               'class' : 'SimpleStrategy', 'replication_factor' : 1 } """)
except Exception as e:
    print(e)

#### Set Keyspace

In [58]:
try:
    session.set_keyspace('music_library_db')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### TASK 1: Getting the artist, song title and its length in music app history

* For this query *sessionId* is used as partition key and *itemInSession* is used as clustering column. 

* Creating the table *song_data* to execute the query for task 1. 

In [59]:
query = "CREATE TABLE IF NOT EXISTS music_library_db.song_data"
query1 = query + "(sessionId int, itemInSession int, \
                        artist text, song text, length float, userId int, \
                        PRIMARY KEY(sessionId ,itemInSession))"
session.execute(query1)                    

* Inserting the values into the table *song_data* 

In [60]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_data (sessionId, itemInSession, \
                        artist, song, length, userId)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]), int(line[10])))

* Verifying the data in the table *song_data*

In [61]:
query_T1 = "SELECT artist, song, length, userId FROM song_data WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query_T1)
except Exception as e:
    print(e)
for row in rows:
        print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### TASK 2: Getting the artist name, song (in itermInSession order) and username(first and last name) in music app history

* For this query *userId* and *sessionId* are used as partition keys and *itemInSession*, *firstName* and *lastName* are used as clustering columns. 

* Creating the table *artist_song* to execute the query for task 2. 

In [62]:
query2 = "CREATE TABLE IF NOT EXISTS music_library_db.artist_song"
query3 = query2 + "(userId int, sessionId int, artist text, \
                            song text, itemInSession int, firstName text, \
                            lastName text, PRIMARY KEY((userId ,sessionId),itemInSession, firstName, lastName))"
session.execute(query3)

* Inserting the values into the table *artist_song*

In [63]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song (userId, sessionId, artist, \
                            song, itemInSession, firstName, lastName)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

* Verifying the data in the table *artist_song*

In [64]:
query_T2 = "SELECT artist, song FROM artist_song WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession, firstName, lastName"

try:
    rows = session.execute(query_T2)
except Exception as e:
    print(e)
for row in rows:
        print(row.artist, row.song)   

Down To The Bone Keep On Keepin' On
Three Drives Greece 2000
Sebastien Tellier Kilometer
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


#### TASK 3: Getting the username(first and last name) for given song in music app history

* For this query *song* is used as partition key and *userId* is used as clustering column. 

* Creating the table *user_name* to execute the query for task 3. 

In [65]:
query4 = "CREATE TABLE IF NOT EXISTS music_library_db.user_name"
query5 = query4 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"
session.execute(query5)

* Inserting the values into the table *user_name*

In [66]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_name (song, userId, firstName, lastName)"
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

* Verifying the data in the table *user_name*

In [69]:
query_T3 = "SELECT firstName, lastName FROM user_name WHERE song = 'All Hands Against His Own' ORDER BY userId;"

try:
    rows = session.execute(query_T3)
except Exception as e:
    print(e)
for row in rows:
        print(row.firstName, row.lastName)

AttributeError: 'Row' object has no attribute 'firstName'

### Dropping  the tables before closing out the sessions

In [48]:
try:
    session.execute("DROP TABLE song_data")
    session.execute("DROP TABLE artist_song")
    session.execute("DROP TABLE user_name")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [49]:
session.shutdown()
cluster.shutdown()